[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hendersonneurolab/CogAI_Fall2025/blob/master/Lab07_Encoding_Model.ipynb)

## Week 7: Build an fMRI encoding model.

In this tutorial, we'll learn how to build a simple encoding model for fMRI data, using data from the Natural Scenes Dataset and features from the AlexNet model. We will fit the model weights using ridge regression, and evaluate its performance on a held-out dataset.

**Learning objectives**
- Understand the ingredients for an fMRI encoding model, including the format of data and feature inputs.
- Know the basic steps involved in fitting a ridge regression model, and how to select the $\lambda$ parameter.
- Be able to interpret metrics related to model performance on held-out data, and how these relate to the reliability of the fMRI data.



In [ ]:
import numpy as np
import urllib.request
from io import BytesIO
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import os, sys
import h5py
import time
import torch
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

### Step 1: Download the data files and features.

Download the data to your google drive here:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Navigate to the Colab Notebooks folder
colab_notebooks_path = '/content/drive/MyDrive/Colab Notebooks/'
os.chdir(colab_notebooks_path)
os.makedirs('CogAI', exist_ok=True)
os.makedirs('CogAI/data', exist_ok=True)
data_folder = os.path.join(colab_notebooks_path, 'CogAI', 'data')
print(data_folder)

In [ ]:
# Info about ROIs
dbox_link = 'https://www.dropbox.com/scl/fi/kilrzj841mrpm17aj9gid/S1_voxel_roi_info.npy?rlkey=jgt1zje70ta8qpmaib8kmjskp&st=n9b3bxft&dl=1'
filename = os.path.join(data_folder, 'S1_voxel_roi_info.npy')
if not os.path.exists(filename):
  st = time.time()
  print('downloading to %s...'%filename)
  urllib.request.urlretrieve(dbox_link, filename)
  print('elapsed = %.2f seconds'%(time.time() - st))
else:
  print('We already have: %s'%filename)

In [ ]:
# Info about images
dbox_link = 'https://www.dropbox.com/scl/fi/caabn3on6l9q8w32uxq8z/S1_image_info.csv?rlkey=cwb4mfruzcyrozrmdrfgerpp2&st=fg9i8ml3&dl=1'
filename = os.path.join(data_folder, 'S1_image_info.csv')
if not os.path.exists(filename):
  st = time.time()
  print('downloading to %s...'%filename)
  urllib.request.urlretrieve(dbox_link, filename)
  print('elapsed = %.2f seconds'%(time.time() - st))
else:
  print('We already have: %s'%filename)

fMRI data: this one can take a while to download, but shouldn't take more than 2 minutes. If it's taking too long, check with the professor.

In [ ]:
# fMRI data
dbox_link = 'https://www.dropbox.com/scl/fi/e040hit5avrptp4hbnijy/S1_betas_avg_bigmask.hdf5?rlkey=s8bpoara1fln1dhf1ydjpuldn&st=323ct1jm&dl=1'
filename = os.path.join(data_folder, 'S1_betas_avg_bigmask.hdf5')
if not os.path.exists(filename):
  st = time.time()
  print('downloading to %s...'%filename)
  urllib.request.urlretrieve(dbox_link, filename)
  print('elapsed = %.2f seconds'%(time.time() - st))
else:
  print('We already have: %s'%filename)

Download DNN features:

In [ ]:

dbox_links = ['https://www.dropbox.com/scl/fi/uw12ee05ecgniq3m814z8/NSD_S1_ims224pix_Conv1_ReLU_maxpool.hdf5?rlkey=hkromug6xybh4ddoqxwml18q9&st=dsoudo6e&dl=1', \
              'https://www.dropbox.com/scl/fi/h9henxdggojf6sdtpai62/NSD_S1_ims224pix_Conv2_ReLU_maxpool.hdf5?rlkey=oqkur18hdnhuyl4hbzjpsqxp4&st=pzuhewly&dl=1', \
              'https://www.dropbox.com/scl/fi/vbeb6ijvsc0qeo6dyo9tu/NSD_S1_ims224pix_Conv3_ReLU_maxpool.hdf5?rlkey=u5ipevvu9yosqnbytm81an0jj&st=y9f0f7yy&dl=1', \
              'https://www.dropbox.com/scl/fi/v01uigo7u41ucpb22ymc4/NSD_S1_ims224pix_Conv4_ReLU_maxpool.hdf5?rlkey=4olepwu5sayo7l6w6t4vum6ul&st=8voneijm&dl=1', \
              'https://www.dropbox.com/scl/fi/mzufg5ucj3ilv323hn9vd/NSD_S1_ims224pix_Conv5_ReLU_maxpool.hdf5?rlkey=gs8nxqnhkg2qwn47i8suklv3o&st=0zn59w87&dl=1']
layers = ['Conv1', 'Conv2', 'Conv3', 'Conv4', 'Conv5']

for link, layer in zip(dbox_links, layers):

  filename = os.path.join(data_folder, 'S1_%s.hdf5'%layer)
  if not os.path.exists(filename):
    st = time.time()
    print('downloading to %s...'%filename)
    urllib.request.urlretrieve(link, filename)
    print('elapsed = %.2f seconds'%(time.time() - st))
  else:
    print('We already have: %s'%filename)

Now that the files are downloaded, we need to load them into Python.


First, load the **image information**. This is a .csv file that contains info about all the images shown.

In [ ]:
info_fn = os.path.join(data_folder, 'S1_image_info.csv')
print(info_fn)
info = pd.read_csv(info_fn)
# this df has [10,000] elements. Each element is 1 unique image.
# it contains info about the images and where they came from (within the MS COCO dataset).
# the rows of this correspond exactly to the features files (which will be loaded below).

image_order = np.array(info['unique_ims'])
n_reps = np.array(info['n_reps'])

Check out what's in "info"

In [ ]:
info


Next, load the **fMRI data**.

The data (betas_avg_bigmask) is organized as: [images x voxels]

Each image was shown multiple times, and these values capture average response to each image.

Keep in mind this is already several steps of preprocessing removed from the raw data. Steps like motion correction have already been performed to improve signal quality.

Single-trial beta weights were already extracted using a GLM analysis (similar to what we did last week).

Data have also been z-scored, within each session, and the beta weights for repetitions of each image have been averaged.

Note also that the voxels in this matrix are not the entire brain. They represent a wide portion of visual cortex, including all voxels with reliable signal.
   


In [ ]:
data_filename = os.path.join(data_folder, 'S1_betas_avg_bigmask.hdf5')
print(data_filename)

t = time.time()
with h5py.File(data_filename, 'r') as data_set:
    values = np.copy(data_set['/betas'])
    data_set.close()
elapsed = time.time() - t
print('Took %.5f seconds to load file'%elapsed)

# Some of these values may be nans, only for some subjects
# this is for subjects who didn't complete all 40 sessions of NSD experiment.
# make sure we remove the nans now.
# for subject 1: we should have all the data, no nans.
good_values = ~np.isnan(values[:,0])
print(values.shape)
print(np.sum(~good_values))

voxel_data = values[good_values,:]
print(voxel_data.shape)

# check that nans are exactly where we expect
# nans happen when n_reps=0
assert(np.all(good_values[n_reps>0]))
assert(np.all(~good_values[n_reps==0]))

Load info about the **ROIs (regions of interest)** in this dataset. Conveniently, the labels for these regions are already provided for us.

In [ ]:
# ROI = region of interest. These are visual areas we want to focus on for analysis.
fn = os.path.join(data_folder, 'S1_voxel_roi_info.npy')
print(fn)
rinfo = np.load(fn, allow_pickle=True).item()
# this is a dictionary that contains information about which voxels our data will include.
# voxel_mask is the whole set of voxels we're focusing on. basically all of visual cortex.
voxel_mask = rinfo['voxel_mask']

Load the **DNN features (activations)**, which will be used to construct the encoding model.

These are features from AlexNet, a large CNN model pre-trained on ImageNet. We pre-computed these features ahead of time, from several different layers of the model.

These features are organized as [n_images x n_features], where the images are in the same order as the fMRI data. So, each row in the features matrices corresponds to one row in the fMRI data matrix.

In [ ]:
fdata = []
for layer in layers:
  filename = os.path.join(data_folder, 'S1_%s.hdf5'%layer)
  print(filename)
  with h5py.File(filename, 'r') as f:
      # Explore the file structure
      print("Keys in file:", list(f.keys()))

      # # Load your data (adjust based on your file structure)
      data = np.array(f['features'])
      fdata += [data]

[f.shape for f in fdata]

Let's explore the ROIs in this dataset. We have ROIs defined in several ways: retinotopic early visual areas, and higher-level category selective regions.

In [ ]:
# Retinotopic ROI definitions:
print(rinfo['ret_prf_roi_names'])
# Face-selective ROI definitions:
print(rinfo['floc_face_roi_names'])
# Place-selective ROI definitions:
print(rinfo['floc_place_roi_names'])
# Body-selective ROI definitions:
print(rinfo['floc_body_roi_names'])

Write a function that returns the set of voxels in any ROI.

In [ ]:

def get_roi_vox(roi_name = 'FFA-2'):

  if roi_name in rinfo['ret_prf_roi_names']:
    ind_num = rinfo['ret_prf_roi_names'][roi_name]
    roi_inds = rinfo['roi_labels_retino'][voxel_mask]==ind_num

  elif roi_name in rinfo['floc_face_roi_names']:
    ind_num = rinfo['floc_face_roi_names'][roi_name]
    roi_inds = rinfo['roi_labels_face'][voxel_mask]==ind_num

  elif roi_name in rinfo['floc_place_roi_names']:
    ind_num = rinfo['floc_place_roi_names'][roi_name]
    roi_inds = rinfo['roi_labels_place'][voxel_mask]==ind_num

  elif roi_name in rinfo['floc_body_roi_names']:
    ind_num = rinfo['floc_body_roi_names'][roi_name]
    roi_inds = rinfo['roi_labels_body'][voxel_mask]==ind_num

  return roi_inds


Focusing on the retinotopic areas for now:

In [ ]:
roi_name = 'V1v' # ventral part of V1.

roi_inds = get_roi_vox(roi_name = roi_name)

print('%s is %d voxels in size'%(roi_name, np.sum(roi_inds)))



---
***Question 1:***

Print out the sizes of the following retinotopic ROIs:
- V1d, V2v, V2d, V3v, V3d, hV4


In [ ]:
# [answer here]



---



Let's focus on one voxel for the next step. We're going to pick one from a target area, and picking the one with the best reliability.

Recall that the **noise ceiling (NC)** is a measure that captures voxel reliability. It captures the proportion of total variance in the voxel's response that can be accounted for by its signal, as opposed to noise. We compute noise ceiling by measuring how much the voxel's response varies in response to repeats of a given image. The idea is, if it varies a lot in response to the same image, it's not very reliable. If it's not reliable, NC is low, and thus our encoding models will not fit well. So NC sets an approximate "ceiling" on encoding model performance.

In [ ]:
# Pick one ROI to focus on here.
roi_name = 'FFA-1'  # (FFA has 2 parts, we'll just grab the first part here.)
roi_inds = get_roi_vox(roi_name = roi_name)
roi_inds_num = np.where(roi_inds)[0]

print('%s has %d voxels'%(roi_name, np.sum(roi_inds)))

# Now getting the FFA voxel with the best noise ceiling.
noise_ceiling = rinfo['noise_ceiling_avgreps'] / 100

# I'm going to choose the "best voxel" in my ROI of interest (highest NC)
# You can also try other voxels and see how this will differ.
best_inds = np.flip(np.argsort(noise_ceiling[roi_inds]))
best_inds = best_inds[0:1]
best_in_roi = roi_inds_num[best_inds]

voxel_inds = best_in_roi

print('best voxel inds are:')
print(voxel_inds)
print('NC of these voxels is:')
print(noise_ceiling[voxel_inds])


### Step 2: Split data into train / test partitions

During ridge regression, we typically split our data into 3 independent sets of images:

- **Training data:** used to fit the model weights
- **Holdout data** (nested validation): used to choose best pRF and ridge parameters
- **Validation data**: held out until the very end, used to compute the validation set $R^2$.


In [ ]:
# fixed random seed, to make sure shuffling is repeatable
rndseeds = [171301]; si = 0

# Always holding out 1000 "shared images", which were seen by all NSD participants, as
# the validation set.
val_inds = np.array(info['shared1000'])

# Then take a random 10% of the remaining data, as the nested "holdout" set.
# Holdout set is used to choose ridge parameters and pRF parameters.
# You could experiment with different % holdout, 10% usually works well.
pct_holdout = 0.10
n_images_total = info.shape[0]
n_images_notval = np.sum(~val_inds);
n_images_holdout = int(np.ceil(n_images_notval*pct_holdout))
n_images_trn = n_images_notval - n_images_holdout

inds_notval = np.where(~val_inds)[0]
np.random.seed(rndseeds[si])
np.random.shuffle(inds_notval) # this is the only random part

inds_trn = inds_notval[0:n_images_trn]
inds_holdout = inds_notval[n_images_trn:]
assert(len(inds_holdout)==n_images_holdout)

trn_inds = np.isin(np.arange(0, n_images_total), inds_trn)
holdout_inds = np.isin(np.arange(0, n_images_total), inds_holdout)

# remove nan rows here
trn_inds = trn_inds[good_values]
val_inds = val_inds[good_values]
holdout_inds = holdout_inds[good_values]

# apply these indices to split the voxel data and image labels.
voxel_data_trn = voxel_data[trn_inds, :]
voxel_data_val = voxel_data[val_inds, :]
voxel_data_holdout = voxel_data[holdout_inds, :]

n_voxels = voxel_data_trn.shape[1]
print(voxel_data_trn.shape, voxel_data_val.shape, voxel_data_holdout.shape)

image_order_use = image_order[good_values]

image_inds_trn = image_order_use[trn_inds]
image_inds_val = image_order_use[val_inds]
image_inds_holdout = image_order_use[holdout_inds]

Now we need to split the model features the same way we split the data.

In this step, we're also going to z-score the features within each column. This is helpful because the different features can have very different variances, and normalizing these variances helps stabilize the resulting fits.


In [ ]:
f = fdata[4] # just grabbing the last Conv layer here. you can try others too...

f_trn = f[trn_inds,:]
f_val = f[val_inds,:]
f_out = f[holdout_inds,:]

# Z-score the data - this is a step that helps with fit stability.
# I'm computing the normalization parameters (mean and std) on my training data only
# (plus the nested held-out partition), but not the val set.
# this helps reduce leakage of data between train and val partitions.
# then apply those same normalization parameters to the val set too.
f_concat = np.concatenate([f_trn, f_out], axis=0)
# f_concat = f_trn

features_m = np.mean(f_concat, axis=0, keepdims=True) #[:trn_size]
# print(features_m[0,0:10])
features_s = np.std(f_concat, axis=0, keepdims=True) + 1e-6

f_trn -= features_m
f_trn /= features_s
f_out -= features_m
f_out /= features_s
f_val -= features_m
f_val /= features_s

# add the intercept: a column of ones
f_trn = np.concatenate([f_trn, np.ones(shape=(len(f_trn), 1), dtype=f_trn.dtype)], axis=1)
f_out = np.concatenate([f_out, np.ones(shape=(len(f_out), 1), dtype=f_out.dtype)], axis=1)
f_val = np.concatenate([f_val, np.ones(shape=(len(f_val), 1), dtype=f_val.dtype)], axis=1)

print('Size of features matrices:')
print(f_trn.shape, f_val.shape, f_out.shape)

Send all numpy arrays to PyTorch tensors.

The main reason we're using torch is because it allows using a GPU for matrix operations.
When we have many voxels to fit, this will be way faster than CPU.
For just a few voxels, this won't take too long even on a CPU.


In [ ]:
# v is our fMRI data, these are the 3 different splits
vtrn = torch.Tensor(voxel_data_trn[:,voxel_inds]).to(device).to(torch.float64)
vout = torch.Tensor(voxel_data_holdout[:,voxel_inds]).to(device).to(torch.float64)
vval = torch.Tensor(voxel_data_val[:,voxel_inds]).to(device).to(torch.float64)

# x is our features, same 3 splits
xtrn = torch.Tensor(f_trn).to(device).to(torch.float64)
xout = torch.Tensor(f_out).to(device).to(torch.float64)
xval = torch.Tensor(f_val).to(device).to(torch.float64)

### Step 3: Solve for weights using ridge regression




First let's set up the candidate $\lambda$ (lambda) values. Recall that lambda is the regularization parameter here. It determines strength of regularization (i.e., how strongly do we push weights to zero).

 Each value in lambdas is a candidate value for the regularizaton strength.

In [ ]:
# lambda is the ridge penalty, bigger = more regularization
n_lambdas = 20
lambdas = np.logspace(np.log(0.0001),np.log(10**8+0.01),n_lambdas, dtype=np.float32, base=np.e) - 0.01

Solve for the weights for each possible lambda. Here we loop over the candidate values, and we get the loss for each one.


The regression problem is:

$$y = X w$$

where:
- $y$ = voxel data
- $X$ = design matrix (features)
- $w$ = regression weights

To solve using ridge regression (L2-regularization):

$$\hat{w} = (X^T X + \lambda I)^{-1} X^T y$$

where:
- $X^T$ = transpose of $X$
- $\lambda$ = regularization parameter
- $I$ = identity matrix
- $(X^T X + \lambda I)^{-1}$ = matrix inverse


**Note:** These equations look a lot like the general linear model (GLM) methdod that we saw last week, used to estimate fMRI responses. The modeling here is a very similar idea to the GLM (except for the addition of the ridge constraint), but it is working with the output of a GLM that has already been run. That is, the fMRI data in Y consists of $\beta$ weights that were extracted using a separate GLM. So we're running a linear model on the output of another linear model.

In [ ]:
losses_per_lambda = np.zeros((n_lambdas, 1))
n_features = xtrn.size()[1]
weights_per_lambda = np.zeros((n_lambdas, n_features))

# loop over all my possible lambda values
for li, ll in enumerate(lambdas):

  # make my identity matrix: this goes into the ridge formula
  identity_matrix = torch.eye(n_features, device=device, dtype=torch.float64)

  # solve for the beta weights
  # computes: w = (X^T X + λI)^(-1) X^T y
  weights = torch.linalg.solve(xtrn.T @ xtrn + ll*identity_matrix, xtrn.T @ vtrn)
  # weights is [n_features x n_voxels]

  # predict the response on held-out data, using features from held-out data (xout)
  pred = xout @ weights
  # pred is [n_images x n_voxels]

  # compute loss for held-out data (sum of squared error)
  # this will tell us the loss for each of the possible lambda values
  loss = torch.sum(torch.pow(vout - pred, 2), dim=0)

  losses_per_lambda[li,:] = loss.item()

  weights_per_lambda[li,:] = weights[:, 0]




---


***Question 2:***

Write some code to find the best lambda value out of the candidate values (lowest loss). Print out the following:
- index of the best lambda
- the lambda value itself
- associated loss for the best lambda
- associated weights for the best lambda



In [ ]:
# [answer here]



---



Now let's visualize a plot of loss versus the regularization strength (lambda). The minimum on this plot is what we want to use.

In [ ]:
loss_array = losses_per_lambda

plt.figure(figsize=(4,3))
plt.plot(lambdas, loss_array, '.-')

plt.title('loss vs ridge params')
plt.xlabel('lambda parameter')
plt.ylabel('Sum of Squared Error')
# plt.yscale('log')
plt.xscale('log')

vi = 0;
best_lambda_ind = np.argmin(losses_per_lambda)
best_lambda = lambdas[best_lambda_ind]

plt.plot(best_lambda, loss_array[best_lambda_ind], 'o', color='r')

### Step 4: Predict responses on held-out data

In [ ]:
# remember that we need to get the weights corresponding to the best lambda value
# (you already found these above)
weights_use = weights_per_lambda[best_lambda_ind,:]

# predict voxel response in held-out validation data here.
# yhat = X @ W
pred = xval @ weights_use[:,None]

# remember to turn these back into numpy, from torch.
# sometimes tensors will give errors in your subsequent numpy code.
actual_array = vval.numpy()
pred_array = pred.numpy()

Now we can evaluate how good the predictions are.

In [ ]:
# first, let's write a function to compute R2
def get_r2(actual,predicted):
    """
    This computes the coefficient of determination (R2).
    Always goes along first dimension (i.e. the trials/samples dimension)
    """
    ssres = np.sum(np.power((predicted - actual),2), axis=0);
    sstot = np.sum(np.power((actual - np.mean(actual, axis=0)),2), axis=0);
    r2 = 1-(ssres/sstot)

    return r2



---
***Question 3:***

Write some code to print out the following:
- R2 for the model on the validation set
- Correlation coefficient for the model on the validation set

In [ ]:
# [answer here]



---



Let's visualize the actual and predicted responses for our example voxel.

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(actual_array[:,0])
plt.plot(pred_array[:,0])

# i'm zooming in on just the first part of this plot, to see better
plt.xlim([0, 200])
plt.legend(['Actual','Predicted'])
plt.xlabel('Image number')
plt.ylabel('Voxel response')

plt.title('Voxel %d, $R^2$ = %.2f'%(voxel_inds[0], get_r2(actual_array, pred_array)))

### Step 5: Run a batch of voxels

Now that we've tested this out for one voxel, let's see how things look across a larger set of voxels.

Use the above function and pull out data from our target ROI.

In [ ]:
# still using FFA here, but can use anything.
target_roi_name = 'FFA-1'

voxel_inds = get_roi_vox(roi_name = target_roi_name)

print('%s has %d voxels'%(target_roi_name, np.sum(voxel_inds)))

# v is our fMRI data, these are the 3 different splits
vtrn = torch.Tensor(voxel_data_trn[:,voxel_inds]).to(device).to(torch.float64)
vout = torch.Tensor(voxel_data_holdout[:,voxel_inds]).to(device).to(torch.float64)
vval = torch.Tensor(voxel_data_val[:,voxel_inds]).to(device).to(torch.float64)


Now, define a function that allows us to solve the ridge problem for many voxels together. The implementation is a bit different than how we did it above, because we want to vectorize the code for speed. But it's doing the same math as we looked at before.

In [ ]:
def solve_ridge_fast(xtrn, vtrn, lambdas):

  n_features = xtrn.shape[1]
  # xT * x
  mult = xtrn.T @ xtrn

  # make an identity matrix
  ridge_term = torch.eye(xtrn.size()[1], device=device, dtype=torch.float64)

  # make versions of this matrix that are adjusted by each possible lambda value
  # this is: (X^T @ X + lambda*I)^(-1)
  # first dim is the different lambda values.
  lambda_matrices = torch.stack([(mult+ridge_term*l).inverse() \
            for l in lambdas], axis=0)

  cofactor = torch.tensordot(lambda_matrices, xtrn, dims=[[2],[1]])

  # solve for weights
  weights = torch.tensordot(cofactor, vtrn, dims=[[2], [0]]) # [#lambdas, #feature, #voxel]

  # predict the response on held-out data, using features from held-out data (xout)
  pred = torch.tensordot(xout, weights, dims=[[1],[1]]) # [#samples, #lambdas, #voxels]

  # compute loss for held-out data
  # this will tell us the loss for each of the possible lambda values
  loss = torch.sum(torch.pow(vout[:,None,:] - pred, 2), dim=0) # [#lambdas, #voxels]

  weights_use = torch.zeros((n_features, vtrn.shape[1]),dtype=weights.dtype)

  # for each voxel, find its best weights
  for vi in range(vtrn.shape[1]):
    # choose the best lambda value, based on min loss
    best_lambda_ind = np.argmin(loss[:,vi])
    best_lambda = lambdas[best_lambda_ind]
    # print(vi, best_lambda_ind, best_lambda)
    weights_use[:, vi] = weights[best_lambda_ind,:,vi]

  return weights_use

Use this function to solve for our ridge weights, for all voxels.

In [ ]:
st = time.time()
weights_use = solve_ridge_fast(xtrn, vtrn, lambdas)
elapsed = time.time() - st
print('elapsed = %.5f s'%elapsed)

Now, predict the held-out data.

In [ ]:
# predict voxel response in held-out validation data here.
# yhat = X @ W
pred = xval @ weights_use

# remember to turn these back into numpy, from torch.
# sometimes tensors will give errors in your subsequent numpy code.
actual_array = vval.numpy()
pred_array = pred.numpy()

Compute R2 for the model fit, for each voxel:

In [ ]:
r2 = get_r2(actual_array, pred_array)


---

***Question 4:***

Print out the maximum, minimum, and median values of $R^2$, across this set of voxels.

In [ ]:
# [answer here]



---



Make a plot of R2 versus the noise ceiling.

In [ ]:
nc = noise_ceiling[voxel_inds]

plt.figure(figsize=(4, 4))
plt.plot(nc, r2,'.')
plt.axhline(0, color=[0.8, 0.8, 0.8])
plt.axvline(0, color=[0.8, 0.8, 0.8])
plt.xlim([-0.1, 1])
plt.ylim([-0.1, 1])
plt.plot([-0.1, 1], [-0.1, 1], '-', color='k')
plt.xlabel('Noise Ceiling')
plt.ylabel('Actual R2')

plt.title('%s'%target_roi_name)



---
***Question 5:***

What relationship do you notice between $R^2$ and the noise ceiling (NC)? Why do they have this relationship? What does this tell us about the overall goodness of the model fit?

[answer here]



---



***Question 6:***

Run the model fitting process for a different ROI. To do this you can copy and paste some of the code from the above sections into the below section and modify it.

For a list of possible ROIs, check back at the top of the code, under the cell "Let's explore the ROIs in this dataset." For example, you could try PPA, or hV4.

Then, do the following:
- Print the min, max, and median $R^2$ for the voxels in this new ROI
- Make a scatter plot of $R^2$ versus noise ceiling (just like the one a few cells above) with your new ROI.
- Interpret this: how do your results compare to the FFA results?


In [ ]:
# [answer here]



---

